In [1]:
!nvidia-smi

Sun Mar 23 13:34:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import pandas as pd
import numpy as np
print("imported")

imported


In [4]:
req_files = ["x_train", "x_test", "y_train", "y_test"]

In [5]:
## url for directory retrieval
base_url = "https://github.com/Subrat1920/IMDB-Sentiment-Analysis/raw/main/Notebook/embeddings/"

In [6]:
## getting the numpy arrays
for file in req_files:
    file_url = f"{base_url}{file}.npy"
    !wget {file_url} -O {file}.npy

--2025-03-23 13:34:13--  https://github.com/Subrat1920/IMDB-Sentiment-Analysis/raw/main/Notebook/embeddings/x_train.npy
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Subrat1920/IMDB-Sentiment-Analysis/main/Notebook/embeddings/x_train.npy [following]
--2025-03-23 13:34:14--  https://raw.githubusercontent.com/Subrat1920/IMDB-Sentiment-Analysis/main/Notebook/embeddings/x_train.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37020128 (35M) [application/octet-stream]
Saving to: ‘x_train.npy’

x_train.npy         100%[===================>]  35.30M  --.-KB/s    in 0.1s    

2025-03-23 13:34:14 (2

In [7]:
## load download the .npy files
x_train = np.load("x_train.npy", allow_pickle=True)
x_test = np.load("x_test.npy", allow_pickle=True)
y_train = np.load("y_train.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

In [8]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5000, 1851), (5000,), (5000, 1851), (5000,))

In [18]:
max_len = 500
max_words = 4995

In [10]:
print("Sample Embeddings")
print("="*50)
print(x_train[0])
print("="*50)
print("And its corresponding sample label")
print("="*50)
print(y_train[0])
print("="*50)

Sample Embeddings
[  0   0   0 ...  19 178  32]
And its corresponding sample label
1


In [25]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, Embedding, Bidirectional
import warnings
warnings.filterwarnings("ignore")

In [26]:
model = Sequential()
model.add(Embedding(max_words, 300, input_length=max_len))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(GRU(64))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, max_len))

In [27]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 500, 300)            │       1,498,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 500, 256)            │         330,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 500, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 64)                  │          61,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,890,629 (7.21 MB)

 Trainable params: 1,890,629 (7.21 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [31]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0005)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [32]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [33]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=early_stopping, validation_data=(x_test, y_test))

Epoch 1/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 34s 356ms/step - accuracy: 0.5532 - loss: 0.6800 - val_accuracy: 0.7140 - val_loss: 0.5691
Epoch 2/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 38s 360ms/step - accuracy: 0.8148 - loss: 0.3972 - val_accuracy: 0.7746 - val_loss: 0.5301
Epoch 3/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 30s 387ms/step - accuracy: 0.8932 - loss: 0.2688 - val_accuracy: 0.8060 - val_loss: 0.4883
Epoch 4/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 38s 354ms/step - accuracy: 0.9539 - loss: 0.1209 - val_accuracy: 0.8096 - val_loss: 0.5800
Epoch 5/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 391ms/step - accuracy: 0.9779 - loss: 0.0697 - val_accuracy: 0.7802 - val_loss: 0.7412
Epoch 6/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 41s 385ms/step - accuracy: 0.9852 - loss: 0.0483 - val_accuracy: 0.7918 - val_loss: 0.8470


In [35]:
model.save("sentiment_model.keras")

In [36]:
from google.colab import files
files.download("sentiment_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>